In [ ]:
import cv2
import tensorflow
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from keras import backend as K
from keras.utils import np_utils, to_categorical
from keras.layers import Dense,Activation,Flatten,Conv2D,MaxPooling2D,Dropout, GlobalAvgPool2D, Softmax
from keras.models import Sequential, load_model
from keras import losses
import keras
import efficientnet.keras as efn
import xgboost as xgb
import albumentations

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

seed= 1
random.seed(seed)
np.random.seed(seed)
tensorflow.set_random_seed(seed)

In [ ]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow
import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    
    return None

In [ ]:
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception


def resnet(lr, label_smooth):
    net= ResNet50V2(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(None,None,3))
    
    model= Sequential()
    model.add(net)
    model.add(GlobalAvgPool2D())
    model.add(Dense(6, activation= 'softmax'))
    
    model.layers[0].trainable= True
    opt= keras.optimizers.Adam(learning_rate= lr)
    loss= losses.CategoricalCrossentropy(label_smoothing= label_smooth, name= 'categorical_crossentropy' )
    model.compile(loss=loss, optimizer= opt, metrics=['accuracy'])
    
    return model



def effnet(lr, label_smooth):
    
    net= efn.EfficientNetB2(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(None,None,3))
    
    model= Sequential()
    model.add(net)
    model.add(GlobalAvgPool2D())
    model.add(Dense(6, activation= 'softmax'))
    
    model.layers[0].trainable= True
    opt = keras.optimizers.Adam(learning_rate= lr)
    loss= losses.CategoricalCrossentropy(label_smoothing= label_smooth, name= 'categorical_crossentropy' )
    model.compile(loss=loss, optimizer= opt, metrics=['accuracy'])
    
    return model




def xception(lr, label_smooth):
    
    net= Xception(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(None,None,3))
    
    model= Sequential()
    model.add(net)
    model.add(GlobalAvgPool2D())
    model.add(Dense(6, activation= None))
    model.add(Softmax())
    
    model.layers[0].trainable= True
    opt = keras.optimizers.Adam(learning_rate= lr)
    loss= losses.CategoricalCrossentropy(label_smoothing= label_smooth, name= 'categorical_crossentropy' )
    model.compile(loss=loss, optimizer= opt, metrics=['accuracy'])
    
    return model

In [ ]:
def data_aug(train_x, train_y):
    
    new_x= []
    new_y= []
    
    transform = albumentations.Compose([
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.Blur(blur_limit=2, p=0.5),
        albumentations.ChannelShuffle(p=0.5),
        #albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30,
                                        interpolation=cv2.INTER_LINEAR, border_mode=2, p=0.5),
    ])
    
    for i in range(len(train_x)):
        
        aug_1= transform(image= train_x[i].astype(np.float32))
        new_x.append(aug_1['image'])
        new_y.append(train_y[i])
        
        
    return np.array(new_x), np.array(new_y)

In [ ]:
def mixup(train_x, train_y, alpha= 0.2, prob= 0.5):
    
    new_x= []
    new_y= []
    
    for i in range(train_x.shape[0]):
        
        if np.random.rand() <= prob: 
            new_x.append(train_x[i])
            new_y.append(train_y[i])
            continue
        
        while True:
            fuse_index= np.random.randint(len(train_x))
            if train_y[fuse_index].argmax(0)!= train_y[i].argmax(0): break
            
        mult= np.random.beta(alpha,alpha)
        
        mix_x= train_x[i]*mult + train_x[fuse_index]*(1-mult)
        mix_y= train_y[i]*mult + train_y[fuse_index]*(1-mult)
        
        new_x.append(mix_x)
        new_y.append(mix_y)
        
        
    return np.array(new_x), np.array(new_y)

In [ ]:
def load_data(img_folder, train_or_test, img_size):
    
    if train_or_test=='train':
    
        folder= os.listdir(img_folder)
        train_x= []
        train_y= []

        
        all_img_name= []
        for f in folder:
            
            folder_img_name= os.listdir(img_folder+'/'+f)
            all_img_name+= folder_img_name
            
            for name in folder_img_name:

                img= cv2.imread(img_folder+'/'+f+'/'+name, 1)
                img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
                img= cv2.resize(img, (img_size,img_size))
                train_x.append(img)
                train_y.append(folder.index(f))
                
                
    else:
        train_x= []
        train_y= []
        all_img_name= os.listdir(img_folder)
        
        for name in all_img_name:
            img= cv2.imread(img_folder+'/'+name, 1)
            img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
            img= cv2.resize(img, (img_size,img_size))
            train_x.append(img)
        
        
    return np.array(train_x), np.array(train_y), np.array(all_img_name)

# Train

In [ ]:
CFG= {
    'model_name': 'effnet',
    'img_size': 128,
    'batch_size': 20,
    'epoch': 30,
    'n_fold': 5,
    'lr': 3e-4,
    'label_smooth': 0.5,
    'load_model': ''#'test_cv_model/xcepnet_8.29',
}

In [ ]:
all_x, all_y, img_name= load_data('Dataset/train_dataset_expand', 'train', CFG['img_size'])
all_y= to_categorical(all_y, 6)
index= list(range(len(all_x)))
np.random.shuffle(index)
all_x= all_x[index]
all_y= all_y[index]

print(all_x.shape)
print(all_y.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits= CFG['n_fold'], shuffle=True, random_state=42)

In [ ]:
for cv, (train_index, vali_index) in enumerate(kf.split(all_x, all_y.argmax(1))):
    #if cv!=0: continue
    print('start cv: ', cv)

    vali_x, vali_y= all_x[vali_index], all_y[vali_index]
    epoch= CFG['epoch']
    
    
    if CFG['load_model']!='':
        print('load_model')
        model= load_model(CFG['load_model']+'/'+'model_cv_'+str(cv)+'.h5')
    else:
        model= effnet(CFG['lr'], CFG['label_smooth'])

        
    best_vali_acc= 0
    for ep in range(epoch):

        aug_x, aug_y= all_x[train_index], all_y[train_index]
        aug_x, aug_y= data_aug(aug_x, aug_y)
        #aug_x, aug_y= mixup(aug_x, aug_y, 0.2)


        y_integers = np.argmax(aug_y, axis=1)
        class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
        d_class_weights = dict(enumerate(class_weights))

        train_history= model.fit(x= aug_x, y= aug_y, validation_data=(vali_x, vali_y), epochs=1, 
                                 batch_size= CFG['batch_size'],
                                 class_weight= d_class_weights,
                                 verbose=1)


        if train_history.history['val_accuracy'][0] > best_vali_acc:
            best_vali_acc= train_history.history['val_accuracy'][0]
            print('model save at vali_acc: ', train_history.history['val_accuracy'][0])
            model.save('train_cv_model/model_cv_'+str(cv)+'.h5')

        del aug_x
        del aug_y
        
        if ep == epoch-15:
            model= load_model('train_cv_model/model_cv_'+str(cv)+'.h5')
            lr= CFG['lr']*0.1
            print('reduce lr: ', lr)
            K.set_value(model.optimizer.learning_rate, lr)
        
    reset_keras()

# Test

In [ ]:
from sklearn.metrics import confusion_matrix


def TTA(model, test_x):
    
    predict= model.predict(test_x)
    
    test_flip= test_x.copy()
    for i in range(len(test_flip)):
        test_flip[i]= np.flip(test_flip[i], 0)
    predict+= model.predict(test_flip)
    del test_flip
    
    test_flip= test_x.copy()
    for i in range(len(test_flip)):
        test_flip[i]= np.flip(test_flip[i], 1)
    predict+= model.predict(test_flip)
    del test_flip
        
    predict= predict/3
    
    return np.array(predict)

In [ ]:
test_CFG= {
    'img_size': 128,
    'load_model': 'train_cv_model'
}

In [ ]:
model_name= os.listdir(test_CFG['load_model'])
model_name= [test_CFG['load_model']+'/'+name for name in model_name]
print(model_name)

test_x, __, img_name= load_data('Dataset/test_images', 'test', test_CFG['img_size'])
test_x.shape

In [ ]:
for i, model in enumerate(model_name):
    
    model= load_model(model)
    ans= TTA(model, test_x)
    #ans= model.predict(test_x)
    
    if i==0:
        pred= ans
    else:
        pred+= ans
    reset_keras()
    
pred/= len(model_name)
pred= pred.argmax(1)
    
submit= pd.read_csv('upload_sample.csv')
for i, name in enumerate(img_name):
    submit.loc[submit['ID']==name, 'Label']= pred[i]
    
submit.to_csv('submit.csv', index= False)

# PL

In [ ]:
def drop_low_conf(test_x, test_y, conf):
    
    drop_index= []
    for i in range(len(test_y)):
        if np.max(test_y[i])<conf:
            drop_index.append(i)
            
    test_x= np.delete(test_x, drop_index, axis=0)
    test_y= np.delete(test_y, drop_index, axis=0)
    
    print('drop '+str(len(drop_index))+' samples')
    
    return test_x, test_y

In [ ]:
test_CFG= {
    'img_size': 128,
    'load_model': 'test_cv_model/effnetb0_128_0.9948',
}


model_name= os.listdir(test_CFG['load_model'])
model_name= [test_CFG['load_model']+'/'+name for name in model_name]
print(model_name)

test_x, __, img_name= load_data('Dataset/test_images', 'test', test_CFG['img_size'])
test_x.shape

In [ ]:
for i, model in enumerate(model_name):
    
    model= load_model(model)
    #ans= TTA(model, test_x)
    ans= model.predict(test_x)
    
    if i==0:
        pred= ans
    else:
        pred+= ans
    reset_keras()
    
pred/= len(model_name)

In [ ]:
test_x, test_y= drop_low_conf(test_x, pred, conf= 0.57)
test_y= test_y.argmax(1)
test_y= to_categorical(test_y, 6)

In [ ]:
CFG= {
    'model_name': 'effnet',
    'img_size': 128,
    'batch_size': 20,
    'epoch': 30,
    'n_fold': 5,
    'lr': 3e-4,
    'label_smooth': 0.5,
    'load_model': ''#'test_cv_model/xcepnet_8.29',
}


all_x, all_y, img_name= load_data('Dataset/train_dataset_expand', 'train', CFG['img_size'])
all_y= to_categorical(all_y, 6)
index= list(range(len(all_x)))
np.random.shuffle(index)
all_x= all_x[index]
all_y= all_y[index]

print(all_x.shape)
print(all_y.shape)


from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits= CFG['n_fold'], shuffle=True, random_state=42)

In [ ]:
all_x, all_y= np.append(all_x, test_x, axis= 0), np.append(all_y, test_y, axis= 0)
del test_x
del test_y


for cv, (train_index, vali_index) in enumerate(kf.split(all_x, all_y.argmax(1))):
    #if cv==0 or cv==1: continue
    print('start cv: ', cv)

    train_x, train_y= all_x[train_index], all_y[train_index]
    vali_x, vali_y= all_x[vali_index], all_y[vali_index]
    epoch= CFG['epoch']
    
    
    if CFG['load_model']!='':
        print('load_model')
        model= load_model(CFG['load_model']+'/'+'model_cv_'+str(cv)+'.h5')
    else:
        model= effnet(CFG['lr'], CFG['label_smooth'])
        #model= resnet(CFG['lr'], CFG['label_smooth'])

        
    best_vali_acc= 0
    for ep in range(epoch):

        aug_x, aug_y= train_x.copy(), train_y.copy()
        aug_x, aug_y= data_aug(aug_x, aug_y)
        #aug_x, aug_y= mixup(aug_x, aug_y, 0.2)


        y_integers = np.argmax(aug_y, axis=1)
        class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
        d_class_weights = dict(enumerate(class_weights))

        train_history= model.fit(x= aug_x, y= aug_y, validation_data=(vali_x, vali_y), epochs=1, 
                                 batch_size= CFG['batch_size'],
                                 class_weight= d_class_weights,
                                 verbose=1)


        if train_history.history['val_accuracy'][0] > best_vali_acc:
            best_vali_acc= train_history.history['val_accuracy'][0]
            print('model save at vali_acc: ', train_history.history['val_accuracy'][0])
            model.save('train_cv_model/model_cv_'+str(cv)+'.h5')

        del aug_x
        del aug_y
        
        if ep == epoch-15:
            model= load_model('train_cv_model/model_cv_'+str(cv)+'.h5')
            lr= CFG['lr']*0.1
            print('reduce lr: ', lr)
            K.set_value(model.optimizer.learning_rate, lr)
        
    reset_keras()

# KD

In [ ]:
def softmax_temp(pred, t= 0.5):
    
    new_y= []
    for ans in pred:
        temp_ans= [np.exp(p/t) for p in ans]
        softmax= [p/np.sum(temp_ans) for p in temp_ans]
        new_y.append(softmax)
    
    return np.array(new_y)

In [ ]:
KD_CFG= {
    'load_model': 'test_cv_model/KD_1'
}

In [ ]:
all_x, all_y, img_name= load_data('Dataset/train_dataset_expand', 'train', CFG['img_size'])
all_y= to_categorical(all_y, 6)
index= list(range(len(all_x)))
np.random.shuffle(index)
all_x= all_x[index]
all_y= all_y[index]
img_name= img_name[index]
new_y= np.zeros_like(all_y)

print(all_x.shape)
print(all_y.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits= CFG['n_fold'], shuffle=True, random_state=42)

In [ ]:
for cv, (train_index, vali_index) in enumerate(kf.split(all_x, all_y.argmax(1))):
    print('start cv: ', cv)

    vali_x, vali_y= all_x[vali_index], all_y[vali_index]
    vali_name= img_name[vali_index]
    
    print('load_model')
    model= load_model(KD_CFG['load_model']+'/'+'model_cv_'+str(cv)+'.h5')
    

    vali_pred= model.predict(vali_x)
    fuse_label= (0.5*vali_y) + (0.5*vali_pred)
    
    
    for i ,name in enumerate(vali_name):
        indx= list(img_name).index(name)
        new_y[indx]= fuse_label[i]
        
    reset_keras()

In [ ]:
CFG= {
    'model_name': 'effnet',
    'img_size': 128,
    'batch_size': 20,
    'epoch': 30,
    'n_fold': 5,
    'lr': 3e-4,
    'label_smooth': 0,
}


all_y= softmax_temp(new_y, t= 0.4)
for cv, (train_index, vali_index) in enumerate(kf.split(all_x, all_y.argmax(1))):
    if cv!=0: continue
    print('start cv: ', cv)

    vali_x, vali_y= all_x[vali_index], all_y[vali_index]
    epoch= CFG['epoch']
    
    
    model= effnet(CFG['lr'], CFG['label_smooth'])

        
    best_vali_acc= 0
    for ep in range(epoch):

        aug_x, aug_y= all_x[train_index], all_y[train_index]
        aug_x, aug_y= data_aug(aug_x, aug_y)
        #aug_x, aug_y= mixup(aug_x, aug_y, 0.2)


        y_integers = np.argmax(aug_y, axis=1)
        class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
        d_class_weights = dict(enumerate(class_weights))

        train_history= model.fit(x= aug_x, y= aug_y, validation_data=(vali_x, vali_y), epochs=1, 
                                 batch_size= CFG['batch_size'],
                                 class_weight= d_class_weights,
                                 verbose=1)


        if train_history.history['val_accuracy'][0] > best_vali_acc:
            best_vali_acc= train_history.history['val_accuracy'][0]
            print('model save at vali_acc: ', train_history.history['val_accuracy'][0])
            model.save('train_cv_model/model_cv_'+str(cv)+'.h5')

        del aug_x
        del aug_y
        
        if ep == epoch-15:
            model= load_model('train_cv_model/model_cv_'+str(cv)+'.h5')
            lr= CFG['lr']*0.5
            print('reduce lr: ', lr)
            K.set_value(model.optimizer.learning_rate, lr)
        
    reset_keras()